In [1]:
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

In [2]:
import transformers

In [4]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

img_url = '/newdata/tarun/datasets/WebVision/GeoILSVRC/asia/test/umbrella/16046885830.jpg' 
raw_image = Image.open(img_url).convert('RGB')



a photo of a group of people sitting under umbrellas


In [34]:
# conditional image captioning
text = "a photo of"
inputs = processor(raw_image, text, return_tensors="pt").to("cuda")

# inputs["max_length"] = 20
# inputs["min_length"] = 3
inputs["return_dict"] = True

out = model(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

In [37]:
out.keys()

odict_keys(['loss', 'decoder_logits', 'image_embeds', 'last_hidden_state'])

In [38]:
out["image_embeds"].shape

torch.Size([1, 577, 768])

In [40]:
raw_image.size

(384, 256)

In [15]:
inputs.keys()

dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'max_length', 'min_length'])

In [42]:
inputs['pixel_values'].shape

torch.Size([1, 3, 384, 384])

In [48]:
model.vision_model.embeddings

BlipVisionEmbeddings(
  (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
)

In [11]:
model.cls

AttributeError: 'BlipForConditionalGeneration' object has no attribute 'cls'

In [14]:
transformers.models.blip.configuration_blip.BlipVisionConfig()

BlipVisionConfig {
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_size": 768,
  "image_size": 384,
  "initializer_factor": 1.0,
  "initializer_range": 1e-10,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "model_type": "blip_vision_model",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "projection_dim": 512,
  "transformers_version": "4.26.1"
}

In [3]:
import json

In [7]:
# from typing import List
# from collections import Counter

def generate_ngrams(text_list, n):
    """
    Generate the number of unique n-grams from a list of strings.

    :param text_list: List of input strings.
    :param n: The number of elements in each n-gram.
    :return: Number of unique n-grams.
    """
    ngrams = []
    for text in text_list:
        words = text.split()
        for i in range(len(words) - n + 1):
            ngram = ' '.join(words[i:i + n])
            ngrams.append(ngram)
    
    unique_ngrams = set(ngrams)
    return len(unique_ngrams)

# # Example usage
# text_list = ["this is a test", "this is another test", "yet another test"]
# bigrams_count = generate_ngrams(text_list, 2)
# trigrams_count = generate_ngrams(text_list, 3)

# bigrams_count, trigrams_count

In [14]:
datasets = ["GeoPlaces", "GeoImnet"]
domains = ["asia", "usa"]

for dataset in datasets:
    for domain in domains:
        print("{},{}".format(dataset, domain))
        meta = json.load(open(f"../metadata/{dataset.lower()}.json"))[f'{domain}_train']['metadata']

        for cap_src in ["llm_cap_llama_13b", "blip2_cap"]:
            caps = [m[cap_src] for m in meta]
            lens = [len(c.split(" ")) for c in caps]
            mean_len = sum(lens)/len(lens)
            bigrams = generate_ngrams(caps, 2)
            trigrams = generate_ngrams(caps, 3)
            print("{}, BiGrams:{}, TriGrams:{}, len:{}\n".format(cap_src, bigrams, trigrams, mean_len))

GeoPlaces,asia
llm_cap_llama_13b, BiGrams:306829, TriGrams:557686, len:10.398668092190999

blip2_cap, BiGrams:55041, TriGrams:137778, len:9.45704012212701

GeoPlaces,usa
llm_cap_llama_13b, BiGrams:464702, TriGrams:854670, len:12.658952332828028

blip2_cap, BiGrams:61278, TriGrams:137890, len:9.367564987928809

GeoImnet,asia
llm_cap_llama_13b, BiGrams:160293, TriGrams:253329, len:9.857949419399901

blip2_cap, BiGrams:34160, TriGrams:80224, len:9.23782776985536

GeoImnet,usa
llm_cap_llama_13b, BiGrams:370761, TriGrams:631261, len:10.902135461047848

blip2_cap, BiGrams:68864, TriGrams:161560, len:9.455496165465954



In [10]:
rawtags = [m["tags"].replace(","," ") + m["caption"] + m["description"] for m in meta]

In [11]:
generate_ngrams(rawtags,2), generate_ngrams(rawtags, 3)

(1509130, 2097825)